# Part 1 (from the previous submission). Please proceed forward to Part 3.

In [1]:
# import sys
# !{sys.executable} -m pip install beautifulsoup4

from bs4 import BeautifulSoup as bs
import requests as rs
import pandas as pd
import numpy as np

### 1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [2]:
# Gathering the data

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
contents = rs.get(url).text

soup = bs(contents)
table = soup.find('table', attrs={'class':'wikitable sortable'})
trs = table.find_all('tr')
cols = ['PostalCode', 'Borough', 'Neighborhood']
rows = []

for tr in trs:
    td = tr.find_all('td')
    row = [ele.text.strip() for ele in td]
    if row:
        rows.append(row)

df = pd.DataFrame(rows, columns=cols)
display(df.head())


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### 2. Only process the cells that have an assigned borough. Ignore cells with a borough that is 'Not assigned'.

In [3]:
df = df[df.Borough != 'Not assigned']
df.reset_index(inplace=True, drop=True)
display(df.head())

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### 3. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [4]:
df['Neighborhood'] = df.apply(lambda row: row['Borough'] if row['Neighborhood'] == 'Not assigned' else row['Neighborhood'], axis=1)
display(df.head())

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### 4. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [5]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).to_frame()
df.reset_index(inplace=True)
display(df.head())

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

The assignment mentions assigning the name of the borough to the neighbourhood after joining the names of the different neighbourhoods in case they are assigned to multiple postal codes but in this case, these steps were flipped. This was done to ensure that each postal code had a neighborhood assigned to it and that combining the names of the different neighbourhoods would not cause 'Not assigned' to be one of the neighborhoods listed. This was important as it would be much more difficult to find the occurrence of a 'Not assigned' and would require reading each entry individually, using regex to find the text, and then figuring out which borough should placed instead of it. Flipping the steps has the same desired result but is done much more easily.

### 6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [6]:
print("\n The shape of the final DataFrame is:", df.shape)


 The shape of the final DataFrame is: (103, 3)


# Part 2 (from the previous submission). Please proceed forward to Part 3.

### Gathering the Geospatial Data

In [7]:
# import sys
# !{sys.executable} -m pip install geopy

# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]


geodata_df = pd.read_csv('https://cocl.us/Geospatial_data')
# display(geodata_df.head())

df = pd.concat([df.set_index('PostalCode'), geodata_df.set_index('Postal Code')], axis=1, join='inner')
df.reset_index(inplace=True)
df.rename(columns={'index': 'PostalCode'}, inplace=True)
display(df.head())

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3 starts here.

In [8]:
# import sys
# !{sys.executable} -m pip install geopy

import folium
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors


place = 'Toronto, Canada'
geo_address = Nominatim(user_agent="toronto_explorer")
locate_address = geo_address.geocode(place)
map_toronto = folium.Map(location=[locate_address.latitude, locate_address.longitude], zoom_start=10)
map_toronto

In [9]:
for _, row in df.iterrows():
    label = '{} ({}), {}'.format(row.PostalCode, row.Neighborhood, row.Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([row.Latitude, row.Longitude], radius=5, popup=label, color='#252191', fill=True, fill_color='#6ecedb', fill_opacity=0.95, parse_html=False).add_to(map_toronto)

map_toronto

### Isolating only the 'Toronto' boroughs

In [10]:
toronto_data = df[df.Borough.str.contains('Toronto')]
toronto_data.reset_index(inplace=True, drop=True)
toronto_data

# Create a map instance
map_toronto = folium.Map(location=[locate_address.latitude, locate_address.longitude], zoom_start=10)

# Add markers for all 38 neighborhoods
for _, row in toronto_data.iterrows():
    label = '{} ({}), {}'.format(row.PostalCode, row.Neighborhood, row.Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([row.Latitude, row.Longitude], radius=5, popup=label, color='#252191', fill=True, fill_color='#6ecedb', fill_opacity=0.95, parse_html=False).add_to(map_toronto) 
    
map_toronto

In [11]:
neighborhood_name = toronto_data.loc[0, 'Neighborhood']
neighborhood_latitude = toronto_data.loc[0, 'Latitude']
neighborhood_longitude = toronto_data.loc[0, 'Longitude']

print("Latitude and longitude of neighborhood '{}' are [{}, {}]".format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude of neighborhood 'The Beaches' are [43.67635739999999, -79.2930312]


### Foursquare API to return top 100 venues within a 500 meter radius

In [12]:
# Foursquare API
CLIENT_ID = '*************************************' # Put Your Client Id
CLIENT_SECRET = '*********************************' # Put You Client Secret 
VERSION = '********'
LIMIT = 30

limit = 100
radius = 500
explore_url_prefix = 'https://api.foursquare.com/v2/venues/explore'
url = '{}?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(explore_url_prefix, CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, limit)

results = rs.get(url).json()
print(results)

{'meta': {'code': 200, 'requestId': '5f040fca20d13b532cfd60a6'}, 'response': {'headerLocation': 'The Beaches', 'headerFullLocation': 'The Beaches, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 4, 'suggestedBounds': {'ne': {'lat': 43.680857404499996, 'lng': -79.28682091449052}, 'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4bd461bc77b29c74a07d9282', 'name': 'Glen Manor Ravine', 'location': {'address': 'Glen Manor', 'crossStreet': 'Queen St.', 'lat': 43.67682094413784, 'lng': -79.29394208780985, 'labeledLatLngs': [{'label': 'display', 'lat': 43.67682094413784, 'lng': -79.29394208780985}], 'distance': 89, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto 

### Getting the venues in each of the different neighbourhoods

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = rs.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)



df = toronto_data
toronto_venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'],longitudes=df['Longitude'], radius=50)

venues = results['response']['groups'][0]['items']
display(venues)

neighborhood_venues = json_normalize(venues)
display(neighborhood_venues)

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4bd461bc77b29c74a07d9282',
   'name': 'Glen Manor Ravine',
   'location': {'address': 'Glen Manor',
    'crossStreet': 'Queen St.',
    'lat': 43.67682094413784,
    'lng': -79.29394208780985,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.67682094413784,
      'lng': -79.29394208780985}],
    'distance': 89,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto ON', 'Canada']},
   'categories': [{'id': '4bf58dd8d48988d159941735',
     'name': 'Trail',
     'pluralName': 'Trails',
     'shortName': 'Trail',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4bd461bc77b2

C:\Users\jyoti\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,...,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.postalCode,venue.venuePage.id
0,e-0-4bd461bc77b29c74a07d9282-0,0,"[{'summary': 'This spot is popular', 'type': '...",4bd461bc77b29c74a07d9282,Glen Manor Ravine,Glen Manor,Queen St.,43.676821,-79.293942,"[{'label': 'display', 'lat': 43.67682094413784...",...,CA,Toronto,ON,Canada,"[Glen Manor (Queen St.), Toronto ON, Canada]","[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",0,[],NaN,NaN
1,e-0-4ad4c062f964a52011f820e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4ad4c062f964a52011f820e3,The Big Carrot Natural Food Market,125 Southwood Dr,NaN,43.678879,-79.297734,"[{'label': 'display', 'lat': 43.678879, 'lng':...",...,CA,Toronto,ON,Canada,"[125 Southwood Dr, Toronto ON M4E 0B8, Canada]","[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",0,[],M4E 0B8,75150878
2,e-0-4b8daea1f964a520480833e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b8daea1f964a520480833e3,Grover Pub and Grub,676 Kingston Rd.,at Main St.,43.679181,-79.297215,"[{'label': 'display', 'lat': 43.67918143494101...",...,CA,Toronto,ON,Canada,"[676 Kingston Rd. (at Main St.), Toronto ON M4...","[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",0,[],M4E 1R4,NaN
3,e-0-4df91c4bae60f95f82229ad5-3,0,"[{'summary': 'This spot is popular', 'type': '...",4df91c4bae60f95f82229ad5,Upper Beaches,NaN,NaN,43.680563,-79.292869,"[{'label': 'display', 'lat': 43.68056321147582...",...,CA,Toronto,ON,Canada,"[Toronto ON, Canada]","[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",0,[],NaN,NaN


In [14]:
venue_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
neighborhood_venues = neighborhood_venues.loc[:, venue_columns]
display(neighborhood_venues)

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Glen Manor Ravine,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",43.676821,-79.293942
1,The Big Carrot Natural Food Market,"[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",43.678879,-79.297734
2,Grover Pub and Grub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",43.679181,-79.297215
3,Upper Beaches,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.680563,-79.292869


### Fixing the names of the columns by splitting on the period

In [15]:
neighborhood_venues.columns = [column.split(".")[-1] for column in neighborhood_venues.columns]
display(neighborhood_venues)

,name,categories,lat,lng
0,Glen Manor Ravine,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",43.676821,-79.293942
1,The Big Carrot Natural Food Market,"[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",43.678879,-79.297734
2,Grover Pub and Grub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",43.679181,-79.297215
3,Upper Beaches,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.680563,-79.292869


### Getting the caregory for a particular venue by formatting and string manipulation

In [16]:
def get_category(row):
    categories_list = row['categories']
    if categories_list:
        return categories_list[0]['name']
    return None

neighborhood_venues['categories'] = neighborhood_venues.apply(get_category, axis=1)
display(neighborhood_venues)

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


### Assigning the venues to the correct neighbourhoods

In [17]:
venues_list = []

for name, lat, lng in zip(toronto_data['Neighborhood'], toronto_data['Latitude'], toronto_data['Longitude']):
    print("Collecting venues for neighborhood:", name)
    
    # Create API request URL
    limit = 100
    radius = 500
    explore_url_prefix = 'https://api.foursquare.com/v2/venues/explore'
    url = '{}?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format( explore_url_prefix, CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, limit)
    
    # Make the request
    neighborhood_venues = rs.get(url).json()["response"]['groups'][0]['items']
    
    # Add relevant info to venues_list
    venues_list.extend([(
        name, lat, lng,
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in neighborhood_venues])

print("\nThe length of the Venues List is:", len(venues_list))


The length of the Venues List is: 1616


In [20]:
toronto_venues = pd.DataFrame(venues_list)
toronto_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
display(toronto_venues.head())

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [21]:
# toronto_onehot = pd.get_dummies(toronto_venues[['Neighbourhood']])
# toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
# display(toronto_grouped)

### One-Hot Encoding for the type of the Venue

In [22]:
toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories


In [23]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
display(toronto_onehot.head())

,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_onehot.insert(0, 'NeighborhoodName', toronto_venues['Neighborhood']) 
display(toronto_onehot.head())

,NeighborhoodName,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
venues_count, categories_count = toronto_onehot.shape
print("So we have {} different venues and {} venue categories.".format(venues_count, categories_count))

So we have 1616 different venues and 234 venue categories.


### Grouping by neighbourhood with mean frequency of occurrence of the different types of venues

In [26]:
toronto_grouped = toronto_onehot.groupby('NeighborhoodName').mean().reset_index()
display(toronto_grouped)

,NeighborhoodName,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.071429,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.015385,0.000000,0.000000,0.015385,0.000000,0.000000,0.015385
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.013514,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.027027
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.028571,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Returning the top 5 types of venues for a particular neighbourhood

In [27]:
num_top_venues = 5
for neighborhood in toronto_grouped['NeighborhoodName']:
    print("------{}------".format(neighborhood))
    temp = toronto_grouped[toronto_grouped['NeighborhoodName'] == neighborhood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

------Berczy Park------
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2      Beer Bar  0.03
3          Café  0.03
4   Cheese Shop  0.03


------Brockton, Parkdale Village, Exhibition Place------
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3    Intersection  0.05
4         Stadium  0.05


------Business reply mail Processing Centre, South Central Letter Processing Plant Toronto------
                venue  freq
0  Light Rail Station  0.10
1         Yoga Studio  0.05
2                Park  0.05
3          Smoke Shop  0.05
4       Burrito Place  0.05


------CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport------
                venue  freq
0      Airport Lounge  0.14
1     Airport Service  0.14
2     Harbor / Marina  0.07
3       Boat or Ferry  0.07
4  Airport Food Court  0.07


------Central Bay Street------
                 venue  freq
0          Coffee Shop 

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['NeighborhoodName']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(
        toronto_grouped.iloc[ind, :], num_top_venues)

display(neighborhoods_venues_sorted)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Restaurant,Café,Cheese Shop,Beer Bar,Department Store,Diner
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Auto Workshop,Comic Shop,Park,Pizza Place,Moving Target,Recording Studio,Restaurant,Burrito Place,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Harbor / Marina,Boat or Ferry,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Airport Gate,Airport Terminal
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Japanese Restaurant,Café,Thai Restaurant,Salad Place,Bar,Burger Joint,Bubble Tea Shop
5,Christie,Grocery Store,Café,Park,Italian Restaurant,Coffee Shop,Nightclub,Diner,Candy Store,Baby Store,Restaurant
6,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Men's Store,Hotel,Mediterranean Restaurant,Bubble Tea Shop
7,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,Cocktail Bar
8,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Sushi Restaurant,Gym,Coffee Shop,Café,Italian Restaurant,Pharmacy,Seafood Restaurant
9,Davisville North,Gym / Fitness Center,Hotel,Park,Breakfast Spot,Food & Drink Shop,Sandwich Place,Department Store,Pizza Place,Convention Center,Distribution Center


### Clustering Toronto into 5 clusters using K-means clustering

In [30]:
# Set number of clusters
k = 5

# Drop the neighborhood name column so that each column contains only the feature set.
toronto_grouped_clustering = toronto_grouped.drop('NeighborhoodName', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# Add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# Merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(toronto_merged.shape)
display(toronto_merged.head())

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Health Food Store,Neighborhood,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Restaurant,Furniture / Home Store,Bookstore,Yoga Studio,Spa,Indian Restaurant
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Fast Food Restaurant,Park,Pub,Liquor Store,Light Rail Station,Sandwich Place,Burrito Place,Restaurant,Italian Restaurant,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Brewery,Gastropub,Bakery,American Restaurant,Yoga Studio,Neighborhood,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Swim School,Bus Line,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop


### Visualizing the Clusters

In [31]:
# Create a map instance
map_toronto = folium.Map(location=[locate_address.latitude, locate_address.longitude], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_toronto)
       
map_toronto

In [32]:
cluster_0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[ [2] + list(range(5, toronto_merged.shape[1]))]]
display(cluster_0)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,0,Trail,Health Food Store,Neighborhood,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
8,"Moore Park, Summerhill East",0,Park,Tennis Court,Trail,Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Deli / Bodega
10,Rosedale,0,Park,Playground,Trail,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[ [2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,1,Park,Swim School,Bus Line,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop


In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[ [2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"The Danforth West, Riverdale",2,Greek Restaurant,Italian Restaurant,Coffee Shop,Ice Cream Shop,Restaurant,Furniture / Home Store,Bookstore,Yoga Studio,Spa,Indian Restaurant
2,"India Bazaar, The Beaches West",2,Fast Food Restaurant,Park,Pub,Liquor Store,Light Rail Station,Sandwich Place,Burrito Place,Restaurant,Italian Restaurant,Fish & Chips Shop
3,Studio District,2,Café,Coffee Shop,Brewery,Gastropub,Bakery,American Restaurant,Yoga Studio,Neighborhood,Sandwich Place,Cheese Shop
5,Davisville North,2,Gym / Fitness Center,Hotel,Park,Breakfast Spot,Food & Drink Shop,Sandwich Place,Department Store,Pizza Place,Convention Center,Distribution Center
6,"North Toronto West, Lawrence Park",2,Coffee Shop,Clothing Store,Sporting Goods Shop,Gym / Fitness Center,Fast Food Restaurant,Diner,Mexican Restaurant,Cosmetics Shop,Park,Chinese Restaurant
7,Davisville,2,Dessert Shop,Sandwich Place,Pizza Place,Sushi Restaurant,Gym,Coffee Shop,Café,Italian Restaurant,Pharmacy,Seafood Restaurant
9,"Summerhill West, Rathnelly, South Hill, Forest...",2,Coffee Shop,Pub,Fried Chicken Joint,Bank,Supermarket,Sushi Restaurant,Bagel Shop,Light Rail Station,Sports Bar,Restaurant
11,"St. James Town, Cabbagetown",2,Restaurant,Coffee Shop,Italian Restaurant,Pizza Place,Park,Market,Bakery,Pub,Café,Japanese Restaurant
12,Church and Wellesley,2,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Yoga Studio,Men's Store,Hotel,Mediterranean Restaurant,Bubble Tea Shop
13,"Regent Park, Harbourfront",2,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Shoe Store


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[ [2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,"Forest Hill North & West, Forest Hill Road Park",3,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[ [2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,4,Garden,Home Service,Pool,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
